# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Create the variable for the file path
csv_path=('../output_data/cities.csv')
# Read the file
weather_data=pd.read_csv(csv_path)
# display the Data
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mahébourg,40,MU,1593577629,93,-20.41,57.70,18.00,1.50
1,Nanortalik,84,GL,1593577605,87,60.14,-45.24,4.53,3.09
2,Pevek,100,RU,1593577831,85,69.70,170.31,3.52,4.21
3,Mandera,77,KE,1593577857,61,3.94,41.87,26.26,9.25
4,Atuona,2,PF,1593577645,74,-9.80,-139.03,26.40,9.76
...,...,...,...,...,...,...,...,...,...
455,Lorengau,69,PG,1593578039,71,-2.02,147.27,29.10,2.58
456,Melilla,20,ES,1593577881,88,35.29,-2.94,23.00,0.50
457,Sinnamary,100,GF,1593578040,89,5.38,-52.95,24.89,2.27
458,Mwingi,59,KE,1593578041,75,-0.93,38.06,19.37,2.56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Configure the map
gmaps.configure(api_key=g_key)
gmaps.figure()
# Display it bigger that default
figure_layout = {
    'width': '900px',
    'height': '900px'
}
# Set the latitude and logitude from the gataframe
coordinates = weather_data[['Lat','Lng']]
# Read the humidity for the heat map
humidity = weather_data['Humidity'].astype(float)
# Apply the layout
fig = gmaps.figure(layout=figure_layout)
# Create the heat map layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=weather_data['Humidity'].max(), point_radius=3)
# Add the layer
fig.add_layer(heat_layer)
# Display the map
fig

Figure(layout=FigureLayout(height='900px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Define the desired parameters for our vacations
upper_temp_desired = 80
lower_temp_desired = 70
wind_speed_desired = 10
cloudiness_desired = 0
# Filter the location that meet all desired parameters
ideal_df = weather_data.loc[(weather_data["Max Temp"] >= 20) & (weather_data["Max Temp"] <= 26) & 
                  (weather_data["Cloudiness"] <= 40) & (weather_data["Humidity"] <= 40)]
# Reset Index
ideal_city_df = ideal_df.reset_index(drop=True)
# Display results
ideal_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Moses Lake,1,US,1593577881,30,47.13,-119.28,21.00,8.20
1,Makat,8,KZ,1593577898,32,47.65,53.35,22.82,6.36
2,Vostok,0,RU,1593577906,36,46.49,135.88,24.48,3.41
3,Jalu,0,LY,1593577831,36,29.03,21.55,24.68,2.67
4,Sakakah,0,SA,1593577625,39,29.97,40.21,26.00,4.60
5,Port Augusta,0,AU,1593577834,25,-32.50,137.77,23.00,11.30
6,Alice Springs,5,AU,1593577999,21,-23.70,133.88,26.00,3.60
7,Mildura,40,AU,1593578017,24,-34.20,142.15,21.11,6.70
8,Narrabri,0,AU,1593577759,40,-30.32,149.78,21.00,4.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Add a new column to store the hotel name
ideal_city_df['Hotel Name'] = ''

In [19]:
# Establish the paramaters from the google API
base_url= 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {
    "radius":5000,
    "rankBy":"Distance",
    "types": "hotel",
    "keyword": "hotel",
    "key":g_key}
# Get data for each latitud and longitud and select the first hotel name and store it in the Hotel Name column
for index, row in ideal_city_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = {f'{lat},{lng}'}
    hotels = requests.get(base_url, params=params).json()
    try:
        ideal_city_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        # If there is no result, a NaN will be added for further cleaning
        ideal_city_df.loc[index, "Hotel Name"] = np.nan


In [20]:
# Clean NaN row that had no hotel
ideal_city_df.dropna(inplace=True)
# Display the Results
ideal_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Moses Lake,1,US,1593577881,30,47.13,-119.28,21.00,8.20,Best Western Plus Lake Front Hotel
1,Makat,8,KZ,1593577898,32,47.65,53.35,22.82,6.36,Gostinitsa Daulet
3,Jalu,0,LY,1593577831,36,29.03,21.55,24.68,2.67,JALO HOTEL
4,Sakakah,0,SA,1593577625,39,29.97,40.21,26.00,4.60,Raoum Inn Hotel
5,Port Augusta,0,AU,1593577834,25,-32.50,137.77,23.00,11.30,Majestic Oasis Apartments
6,Alice Springs,5,AU,1593577999,21,-23.70,133.88,26.00,3.60,DoubleTree by Hilton Hotel Alice Springs
7,Mildura,40,AU,1593578017,24,-34.20,142.15,21.11,6.70,Quality Hotel Mildura Grand
8,Narrabri,0,AU,1593577759,40,-30.32,149.78,21.00,4.10,Kaputar Motel Narrabri


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_df.iterrows()]
locations = ideal_city_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)
fig


Figure(layout=FigureLayout(height='900px', width='900px'))